In [2]:
# Import necessary libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.callbacks import ModelCheckpoint
import numpy as np
import os


In [3]:
# Set the paths to your dataset
train_path = './chest_xray/train'
test_path = './chest_xray/test'
val_path = './chest_xray/val'

# Set the image dimensions
img_width, img_height = 150, 150


In [4]:
# Create data generators for training, testing, and validation sets
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss=categorical_crossentropy, metrics=['accuracy'])

c:\Users\MUNEER IQBAL\.conda\envs\tensorflow_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
# Train the model
model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    callbacks=[ModelCheckpoint('model.h5', save_best_only=True)]
)

Epoch 1/5
163/163 [==============================] - 219s 1s/step - loss: 1.2940 - accuracy: 0.8643 - val_loss: 0.4589 - val_accuracy: 0.6875
Epoch 2/5
163/163 [==============================] - 195s 1s/step - loss: 0.0822 - accuracy: 0.9745 - val_loss: 0.2081 - val_accuracy: 0.9375
Epoch 3/5
163/163 [==============================] - 183s 1s/step - loss: 0.0521 - accuracy: 0.9822 - val_loss: 0.1431 - val_accuracy: 0.9375
Epoch 4/5
163/163 [==============================] - 179s 1s/step - loss: 0.0427 - accuracy: 0.9858 - val_loss: 0.1997 - val_accuracy: 0.8750
Epoch 5/5
163/163 [==============================] - 186s 1s/step - loss: 0.0219 - accuracy: 0.9941 - val_loss: 0.2649 - val_accuracy: 0.8750


In [7]:
# Save the model
model.save('model2.h5')

import pickle

# Assuming 'model' is your trained deep learning model object
with open('model2.pkl', 'wb') as f:
    pickle.dump(model, f)




INFO:tensorflow:Assets written to: ram://dec36476-b710-4653-ade4-d3ca6f193038/assets


INFO:tensorflow:Assets written to: ram://dec36476-b710-4653-ade4-d3ca6f193038/assets


In [8]:
# Load the saved model
import cv2
model.load_weights('model2.h5')

# Load the new image
new_image = cv2.imread('img.jpg')

# Preprocess the image
new_image = cv2.resize(new_image, (150, 150))
new_image = new_image / 255.0

# Prepare the image for prediction
new_image = np.expand_dims(new_image, axis=0)
new_image = np.expand_dims(new_image, axis=3)

# Make predictions
predictions = model.predict(new_image)

# Convert the predictions to class labels
new_labels = np.argmax(predictions, axis=1)

# Classify the image based on the labels
if new_labels[0] == 0:
    print("The image is not affected by pneumonia.")
else:
    print("The image is affected by pneumonia.")

ValueError: in user code:

    File "c:\Users\MUNEER IQBAL\.conda\envs\tensorflow_env\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\MUNEER IQBAL\.conda\envs\tensorflow_env\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\MUNEER IQBAL\.conda\envs\tensorflow_env\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\MUNEER IQBAL\.conda\envs\tensorflow_env\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "c:\Users\MUNEER IQBAL\.conda\envs\tensorflow_env\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\MUNEER IQBAL\.conda\envs\tens